In [1]:
import cv2 as cv
import numpy as np
import os
import queue

In [2]:
# folder_path reprezinta calea catre folderul ce contine imagini cu sudoku jigsaw
folder_path = "./CAVA-2021-TEMA1/testare/jigsaw"
# digits_path reprezinta calea catre folderul care contine imagini cu cifre, folosite pentru recunoasterea caracterelor
digits_path = "./cifre/jigsaw/"

In [3]:
# creez folderul in care voi scrie rezultatele
new_path = "./rezultate_Vulpoiu_Alexandru"
classic_path = "./rezultate_Vulpoiu_Alexandru/clasic"
jigsaw_path = "./rezultate_Vulpoiu_Alexandru/jigsaw"

if not os.path.exists(new_path):
    os.makedirs(new_path)
if not os.path.exists(classic_path):
    os.makedirs(classic_path)
if not os.path.exists(jigsaw_path):
    os.makedirs(jigsaw_path)

In [4]:
def preprocess_image(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(image, 5)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    _, thresh = cv.threshold(image_sharpened, 30, 255, cv.THRESH_BINARY)

    kernel = np.ones((5, 5), np.uint8)
    thresh = cv.erode(thresh, kernel)
    
    edges = cv.Canny(thresh, 150, 400)
    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    # am modificat imaginea pentru a identifica conturul exterior
    
    max_area = 0
    for i in range(len(contours)):
        if(len(contours[i]) > 3):
            possible_top_left = None
            possible_bottom_right = None
            
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1]:
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis=1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            # am identificat cele 4 puncte care sunt colturile careului
            
            # conturul cu arie maxima va fi si cel exterior
            if cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left
    
    # calculez mijlocul careului sudoku
    sudoku_middle_diag_1 = (top_left + bottom_right) // 2
    sudoku_middle_diag_2 = (top_right + bottom_left) // 2
    sudoku_middle = (sudoku_middle_diag_1 + sudoku_middle_diag_2) // 2
    
    # marchez colturile si mijlocul patratului
    image_copy = cv.cvtColor(image.copy(), cv.COLOR_GRAY2BGR)
    cv.circle(image_copy, tuple(top_left), 4, (0, 0, 255), -1)
    cv.circle(image_copy, tuple(top_right), 4, (0, 0, 255), -1)
    cv.circle(image_copy, tuple(bottom_left), 4, (0, 0, 255), -1)
    cv.circle(image_copy, tuple(bottom_right), 4, (0, 0, 255), -1)
    cv.circle(image_copy, tuple(sudoku_middle), 4, (0, 0, 255), -1)
    
    return top_left, top_right, bottom_left, bottom_right, sudoku_middle

In [5]:
digit_patterns = [] # incarc imaginile cu cifre
for i in range(1, 10): # cifre pentru careurile color
    path = digits_path + "0" + str(i) + "_1.jpg"
    img = cv.imread(path)
    digit_patterns.append(img)

for i in range(1, 10): # cifre pentru careurile alb-negru
    path = digits_path + "0" + str(i) + "_2.jpg"
    img = cv.imread(path)
    digit_patterns.append(img)

cropped_images = [] # lista cu imagini care contin doar careul sudoku, de dimensiune 500 x 500
for idx in range(1, 41):
    if idx < 10:
        number = "0" + str(idx)
    else:
        number = str(idx)
    
    # pentru fiecare imagine, obtin punctele care reprezinta colturile si mijlocul careului
    img = cv.imread(folder_path + "/" + number + ".jpg")
    img = cv.resize(img,(0,0),fx=0.2,fy=0.2)
    top_left, top_right, bottom_left, bottom_right, sudoku_middle = preprocess_image(img)
    
    # calculez diferentele intre punctele de sus, de jos, din stanga si din dreapta
    top_difference = abs(top_right - top_left)
    left_difference = abs(top_left - bottom_left)
    bottom_difference = abs(bottom_left - bottom_right)
    right_difference = abs(bottom_right - top_right)
    
    # calculez unghiurile de inclinare pe fiecare dintre cele 4 laturi
    angle_top = np.arctan(top_difference[1] / top_difference[0])
    angle_left = np.arctan(left_difference[0] / left_difference[1])
    angle_bottom = np.arctan(bottom_difference[1] / bottom_difference[0])
    angle_right = np.arctan(right_difference[0] / right_difference[1])
    
    # calculez unghiul cu care trebuie rotita imaginea pentru a o indrepta
    angle_rad = np.mean(np.array([angle_top, angle_left, angle_bottom, angle_right]))
    angle = np.degrees(angle_rad)
    
    image_copy = img.copy()
    (h, w) = image_copy.shape[: 2]
    
    if top_left[1] >= top_right[1]: # imagine rotita spre stanga = in sens trigonometric
        if (angle_bottom > angle_top and bottom_right[1] > bottom_left[1]) or \
            (angle_left > angle_top and top_left[0] > bottom_left[0]) or \
            (angle_right > angle_top and top_right[0] > bottom_right[0]): # verific daca trebuie sa rotesc tot spre stanga
            # adica daca unghiul de jos/dreapta/stanga este mai mare si are orientare diferita fata de unghiul de sus
            M = cv.getRotationMatrix2D((int(sudoku_middle[0]), int(sudoku_middle[1])), angle, 1.0)
            rotated = cv.warpAffine(image_copy, M, (w, h), borderValue=(255, 255, 255))
        else:
            M = cv.getRotationMatrix2D((int(sudoku_middle[0]), int(sudoku_middle[1])), -angle, 1.0)
            rotated = cv.warpAffine(image_copy, M, (w, h), borderValue=(255, 255, 255))

    else: # imagine rotita spre dreapta, procedez asemanator cu cazul anterior
        if (angle_bottom > angle_top and bottom_left[1] > bottom_right[1]) or \
            (angle_left > angle_top and bottom_left[0] > top_left[0]) or \
            (angle_right > angle_top and bottom_right[0] > top_right[0]): # verific daca trebuie sa rotesc tot spre dreapta
           
            M = cv.getRotationMatrix2D((int(sudoku_middle[0]), int(sudoku_middle[1])), -angle, 1.0)
            rotated = cv.warpAffine(image_copy, M, (w, h), borderValue=(255, 255, 255))
        else:
            M = cv.getRotationMatrix2D((int(sudoku_middle[0]), int(sudoku_middle[1])), angle, 1.0)
            rotated = cv.warpAffine(image_copy, M, (w, h), borderValue=(255, 255, 255))
    
    # rotated este imaginea rotita in jurul centrului careului pentru a ajunge in pozitia normala
    # pentru aceasta imagine, calculez punctele care reprezinta colturile
    top_left, top_right, bottom_left, bottom_right, _ = preprocess_image(rotated)
    
    start_row = (top_left[1] + top_right[1]) // 2
    end_row = (bottom_left[1] + bottom_right[1]) // 2
    start_col = (top_left[0] + bottom_left[0]) // 2
    end_col = (top_right[0] + bottom_right[0]) // 2
    
    # decupez din imagine doar careul sudoku
    img_crop = rotated[start_row: end_row, start_col: end_col].copy()
    img_crop = cv.resize(img_crop, (500, 500))
    cropped_images.append(img_crop)

In [6]:
lines_vertical = []
for i in range(0, 500, 55):
    l = []
    l.append((i, 0))
    l.append((i, 499))
    lines_vertical.append(l)

In [7]:
lines_horizontal = []
for i in range(0, 500, 55):
    l = []
    l.append((0, i))
    l.append((499, i))
    lines_horizontal.append(l)

In [8]:
for img_crop in cropped_images: # desenez linii verticale si orizontale pe imagine pentru a observa delimitarea celulelor
    img_copy = img_crop.copy()
    for line in lines_vertical: 
        cv.line(img_copy, line[0], line[1], (0, 255, 0), 2)
    for line in lines_horizontal: 
        cv.line(img_copy, line[0], line[1], (0, 255, 0), 2)

In [9]:
def modify_patch(image): # modific fiecare patch pana ajung la o imagine cu pixeli albi si negri
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(image, 3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    _, thresh = cv.threshold(image_sharpened, 30, 255, cv.THRESH_BINARY)
    
    # pentru a verifica daca celula este goala, fac media imaginii; daca media este 255, inseamna ca am doar pixeli albi
    patch_mean = np.mean(thresh)
    if patch_mean < 254:
        return 'x'
    else:
        return 'o'

In [10]:
def modify_image(img): # modific imaginile astfel incat la final sa contina doar pixeli albi si negri
    image = img.copy()
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(image, 3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.85, 0)
    _, thresh = cv.threshold(image_sharpened, 30, 255, cv.THRESH_BINARY)
    
    return thresh

In [11]:
def get_results(idx,lines_horizontal,lines_vertical): # obtin configuratia careului sudoku (cu cifre si 'o')
    img_crop = cropped_images[idx - 1]    
    config = [['' for _ in range(9)] for _ in range(9)]
    
    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            
            # decupez cate 15 pixeli din fiecare parte ca sa ma asigur ca nu mai raman si bucati din liniile 
            # care apar in sudoku(decuparea nu va fi mereu perfecta)
            patch = img_crop[x_min + 15: x_max - 15, y_min + 15: y_max - 15].copy()
            value = modify_patch(patch)
            
            if value == 'o':
                config[i][j] = value
            else: # daca patch-ul nu este gol, detectez cifra pe care o contine
                
                # noul patch este celula initiala din care decupez cate 3 pixeli, ca sa elimin eventuale linii ramase
                patch = img_crop[x_min + 3: x_max - 3, y_min + 3: y_max - 3].copy()
                patch_modified = modify_image(patch) # patch-ul curent contine doar pixeli albi si negri
                
                predictions_1 = []
                for digit_index in range(9):
                    img = digit_patterns[digit_index].copy()
                    pattern = modify_image(img) # imaginea curenta cu cifra contine doar pixeli albi si negri
                    
                    # compar patch-ul curent cu cifra curenta
                    result = cv.matchTemplate(patch_modified, pattern, cv.TM_CCOEFF_NORMED)
                    result = result.flatten()
                    
                    # adaug valoarea maxima de matching intr-o lista
                    poz = np.argmax(result)
                    predictions_1.append(result[poz])
                
                # predictions_1 contine pe pozitia i scorul de asemanare cu cifra i + 1 din setul de cifre pentru sudoku color
                predictions_1 = np.asarray(predictions_1)
                # calculez care cifra din setul color se aseamana cel mai mult cu cea din patch
                best_1 = np.argmax(predictions_1)
                
                predictions_2 = []
                for digit_index in range(9, 18):
                    img = digit_patterns[digit_index].copy()
                    pattern = modify_image(img) # imaginea curenta cu cifra contine doar pixeli albi si negri
                    
                    # compar patch-ul curent cu cifra curenta
                    result = cv.matchTemplate(patch_modified, pattern, cv.TM_CCOEFF_NORMED)
                    result = result.flatten()
                    
                    # adaug valoarea maxima de matching intr-o lista
                    poz = np.argmax(result)
                    predictions_2.append(result[poz])
                
                # predictions_2 contine pe pozitia i scorul de asemanare cu cifra i + 1 din setul de cifre pentru sudoku alb-negru
                predictions_2 = np.asarray(predictions_2)
                # calculez care cifra din setul alb-negru se aseamana cel mai mult cu cea din patch
                best_2 = np.argmax(predictions_2)
                
                # pastrez predictia cu scorul de asemanare mai mare dintre maximele pentru color si alb-negru
                if predictions_1[best_1] > predictions_2[best_2]:
                    best = best_1
                else:
                    best = best_2
                config[i][j] = str(best + 1)
            
    return config

In [12]:
def find_groups(idx) -> list: # functia calculeaza grupurile de celule din fiecare sudoku
    group = [['0' for _ in range(9)] for _ in range(9)]
    
    # modific imaginea pentru a pune in evidenta liniile care separa grupurile
    image = cropped_images[idx - 1].copy()
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(image, 7)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 7)
    image_sharpened = cv.addWeighted(image_m_blur, 1.4, image_g_blur, -0.95, 0)
    _, thresh = cv.threshold(image_sharpened, 30, 255, cv.THRESH_BINARY)
    # obtin o imagine alb-negru
    
    # aplic dilatare pentru a ramane in evidenta muchiile de separare
    kernel = np.ones((3, 3), np.uint8)
    thresh = cv.dilate(thresh, kernel)
    
    # desenez contururile pe imagine
    contours, _ = cv.findContours(thresh, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    new_img = cv.drawContours(thresh, contours, -1, color=(0, 255, 0), thickness=1)
    
    sep_between_lines = [[False for _ in range(9)] for _ in range(9)]
    # sep_between_lines[i][j] = True daca linia i este separata de linia i + 1 pe coloana j; False altfel
    sep_between_columns = [[False for _ in range(9)] for _ in range(9)]
    # sep_between_columns[i][j] = True daca coloana j este separata de coloana j + 1 pe linia i; False altfel
    
    # fiecare posibila linie de separare are coordonate multiplii de 55 (imaginea are h = w = 500, deci un patch are h = w = 55)
    for i in range(55, 55 * 8 + 1, 55):
        for j in range(0, 55 * 8 + 1, 55):
            for k in range(-15, 16): # k este o marja de erore de 15 pixeli (liniile pot fi deviate, decuparea nu iese mereu perfect)
                s = np.sum(new_img[j: j + 55, i + k]) # calculez suma pixelilor de pe linia curenta
                if s <= 255 * 10: # daca suma are maxim 10 pixeli albi, inseamna ca sunt minim 45 de pixeli negri, deci avem separare intre coloane
                    sep_between_columns[j // 55][i // 55 - 1] = True
                    break
            
            # analog determinam separarea intre linii
            for k in range(-15, 16):
                s = np.sum(new_img[i + k, j: j + 55])                
                if s <= 255 * 10:
                    sep_between_lines[i // 55 - 1][j // 55] = True
                    break
    
    gr: int = 1 # indexul grupului curent
    for i in range(9):
        for j in range(9):
            if group[i][j] == '0':
                q = queue.Queue()
                q.put((i, j))
                group[i][j] = str(gr)
                
                # determin grupurile aplicand algoritmul lui Lee, plecand din fiecare celula careia nu i-a fost atribuit un grup inca
                while not q.empty():
                    x, y = q.get()
                    
                    # verific daca vecinii nu au grup si daca nu sunt separati de celula curenta
                    if x >= 1 and group[x - 1][y] == '0' and not sep_between_lines[x - 1][y]:
                        q.put((x - 1, y))
                        group[x - 1][y] = str(gr)
                    if y <= 7 and group[x][y + 1] == '0' and not sep_between_columns[x][y]:
                        q.put((x, y + 1))
                        group[x][y + 1] = str(gr)
                    if x <= 7 and group[x + 1][y] == '0' and not sep_between_lines[x][y]:
                        q.put((x + 1, y))
                        group[x + 1][y] = str(gr)
                    if y >= 1 and group[x][y - 1] == '0' and not sep_between_columns[x][y - 1]:
                        q.put((x, y - 1))
                        group[x][y - 1] = str(gr)
                gr += 1
    
    return group

In [13]:
# scriu in fisiere rezultatele pentru fiecare imagine
for idx in range(1, 41):
    groups = find_groups(idx) # determin grupurile pentru celule
    config = get_results(idx, lines_horizontal, lines_vertical) # determin configuratia cu cifre si 'o'
    
    # pentru raspuns, combin grupurile si configuratia
    answer = []
    for i in range(9):
        line = ""
        for j in range(9):
            line += groups[i][j] + config[i][j]
        answer.append(line)
    
    output_file_path = jigsaw_path + "/"
    if idx <= 9:
        output_file_path += "0" + str(idx)
    else:
        output_file_path += str(idx)
    output_file_path += "_bonus_predicted.txt"
    
    output_file = open(output_file_path, "w")
    for i in range(8):
        output_file.write(answer[i] + "\n")
    output_file.write(answer[8])
    
    output_file.close()  